# Explore & Clean the Data

In [1]:
import glob 
import pandas as pd
import uuid

In [2]:
# Get a list of all CSV files in the specified directory
files = glob.glob("../data/JC-2016*.csv")
# Read all files and concatenate them into a single DataFrame
df_list = list()
# Loop through each file and read it into a DataFrame
for file in files:
    try:
        temp_df = pd.read_csv(file)
        df_list.append(temp_df)
    except pd.errors.EmptyDataError:
        print(f'File {file} is empty and will be skipped.')

# Concatenate all DataFrames into a single DataFrame
citibike_df = pd.concat(df_list, ignore_index=True)

In [3]:
print(f"Number of rows in the DataFrame: {citibike_df.shape[0]}\nNumber of columns in the DataFrame: {citibike_df.shape[1]}")

Number of rows in the DataFrame: 247584
Number of columns in the DataFrame: 15


In [4]:
description_df = pd.DataFrame({
    "attribute": citibike_df.columns,
    "null_values": citibike_df.isnull().sum(),
    "number_of_rows": citibike_df.shape[0],
    "data_type": citibike_df.dtypes,
    "unique_values": citibike_df.nunique(),
    "sample_values": [citibike_df[col].dropna().unique()[:5] for col in citibike_df.columns],
        
})

description_df

,attribute,null_values,number_of_rows,data_type,unique_values,sample_values
Trip Duration,Trip Duration,0,247584,int64,6024,"[362, 200, 202, 248, 903]"
Start Time,Start Time,0,247584,object,244407,"[2016-01-01 00:02:52, 2016-01-01 00:18:22, 201..."
Stop Time,Stop Time,0,247584,object,244137,"[2016-01-01 00:08:54, 2016-01-01 00:21:42, 201..."
Start Station ID,Start Station ID,0,247584,int64,51,"[3186, 3209, 3195, 3211, 3187]"
Start Station Name,Start Station Name,0,247584,object,51,"[Grove St PATH, Brunswick St, Sip Ave, Newark ..."
Start Station Latitude,Start Station Latitude,0,247584,float64,51,"[40.71958611647166, 40.7241765, 40.73074262530..."
Start Station Longitude,Start Station Longitude,0,247584,float64,51,"[-74.04311746358871, -74.0506564, -74.06378388..."
End Station ID,End Station ID,0,247584,int64,102,"[3209, 3213, 3203, 3210, 3214]"
End Station Name,End Station Name,0,247584,object,102,"[Brunswick St, Van Vorst Park, Hamilton Park, ..."
End Station Latitude,End Station Latitude,0,247584,float64,102,"[40.7241765, 40.71848892, 40.727595966, 40.742..."


In [ ]:
# Remove rows where 'User Type' is NaN
citibike_df.dropna(subset=['User Type'], inplace=True)

# Calculate the mean birth year 
mean_birth_year = int(citibike_df['Birth Year'].mean())

# Fill NaN values in 'Birth Year' with the mean birth year
citibike_df['Birth Year'] = citibike_df['Birth Year'].fillna(mean_birth_year)

# Convert 'Trip Duration' from seconds to minutes and round to 2 decimal places
citibike_df['Trip Duration Minutes'] = (citibike_df['Trip Duration'] / 60).round(2)

# Map the 'Gender' values to more descriptive names
citibike_df['Gender Name'] = citibike_df['Gender'].map({
    0: 'Unknown',
    1: 'Male',
    2: 'Female'
})

# Map the user type values to numeric IDs
citibike_df['User Type ID'] = citibike_df['User Type'].map({
    'Subscriber': 1,
    'Customer': 2
})

In [6]:
citibike_df.columns

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender', 'Trip Duration Minutes', 'Gender Name',
       'User Type ID'],
      dtype='object')

# Create a DB schema structure - Denormalize

In [7]:
# Create a new column 'Trip Id' starting from 1
citibike_df['Trip Id'] = [str(uuid.uuid4()) for _ in range(len(citibike_df))]

## Extract Trips data

In [8]:
trips_df = citibike_df[['Trip Id', 'Start Time', 'Stop Time', 'Trip Duration Minutes', 
                        'Trip Duration','Start Station ID', 'End Station ID', 
                        'Bike ID', 'User Type ID', 'Birth Year','Gender']]

trips_df = trips_df.rename(columns={'Gender': 'Gender ID', 'Trip Duration': 'Trip Duration Seconds'})

# Standardize column names: lowercase and replace spaces with underscores
trips_df.columns = trips_df.columns.str.lower().str.replace(' ', '_')

# Convert 'Birth Year' to integer type
trips_df['birth_year'] = trips_df['birth_year'].astype(int)

trips_df

,trip_id,start_time,stop_time,trip_duration_minutes,trip_duration_seconds,start_station_id,end_station_id,bike_id,user_type_id,birth_year,gender_id
0,d46c1810-199d-437e-8034-806a559d36ba,2016-01-01 00:02:52,2016-01-01 00:08:54,6.03,362,3186,3209,24647,1,1964,2
1,aecb0eb6-bc3b-4e0e-ac83-ee973b86db62,2016-01-01 00:18:22,2016-01-01 00:21:42,3.33,200,3186,3213,24605,1,1962,1
2,01d03898-956f-41c9-949b-a437edf1849d,2016-01-01 00:18:25,2016-01-01 00:21:47,3.37,202,3186,3213,24689,1,1962,2
3,26cb1959-65b5-4d0b-8026-c7aa6caa7070,2016-01-01 00:23:13,2016-01-01 00:27:21,4.13,248,3209,3203,24693,1,1984,1
4,f1eeeb28-e966-4cec-a2ba-f3233fbfed35,2016-01-01 01:03:20,2016-01-01 01:18:24,15.05,903,3195,3210,24573,2,1979,0
...,...,...,...,...,...,...,...,...,...,...,...
247579,61dc0a69-048a-4b35-ba26-cd26e9125c26,2016-12-31 23:10:16,2016-12-31 23:19:33,9.28,557,3214,3203,24465,1,1981,2
247580,db4db90c-de4d-4813-ab46-2ce71e8b0ce7,2016-12-31 23:29:39,2017-01-01 00:15:29,45.82,2749,3183,3183,24389,2,1979,0
247581,ec376dda-9d61-4eac-a8d4-1b53d4683de2,2016-12-31 23:44:37,2016-12-31 23:47:31,2.88,173,3186,3270,24641,1,1978,1
247582,54ecae1b-4fe9-4449-b3fa-83457db692d7,2016-12-31 23:44:50,2017-01-01 00:25:14,40.40,2424,3214,3214,26219,1,1960,2


## Extract stations data

In [9]:
start_stations = citibike_df[['Start Station ID', 'Start Station Name', 
                              'Start Station Latitude', 'Start Station Longitude']]
start_stations.columns = ['Station ID', 'Name', 'Latitude', 'Longitude']

end_stations = citibike_df[['End Station ID', 'End Station Name', 
                            'End Station Latitude', 'End Station Longitude']]
end_stations.columns = ['Station ID', 'Name', 'Latitude', 'Longitude']

stations_df = pd.concat([start_stations, end_stations]).drop_duplicates('Station ID')

stations_df.columns = stations_df.columns.str.lower().str.replace(' ', '_')

stations_df.head()

,station_id,name,latitude,longitude
0,3186,Grove St PATH,40.719586,-74.043117
3,3209,Brunswick St,40.724176,-74.050656
4,3195,Sip Ave,40.730743,-74.063784
7,3211,Newark Ave,40.721525,-74.046305
8,3187,Warren St,40.721124,-74.038051


## Extract bikes data

In [10]:
bikes_df = citibike_df[['Bike ID']].drop_duplicates().sort_values(by='Bike ID').reset_index(drop=True)
bikes_df.columns = bikes_df.columns.str.lower().str.replace(' ', '_')
bikes_df.head()

,bike_id
0,14552
1,14632
2,14705
3,14717
4,14786


## Extract gender data

In [11]:
gender_df = citibike_df[['Gender', 'Gender Name']].drop_duplicates().sort_values(by='Gender').reset_index(drop=True)
gender_df = gender_df.rename(columns={'Gender': 'Gender ID'})
# Standardize column names: lowercase and replace spaces with underscores
gender_df.columns = gender_df.columns.str.lower().str.replace(' ', '_')
gender_df.head()

,gender_id,gender_name
0,0,Unknown
1,1,Male
2,2,Female


## Extract User data

In [12]:
users_df = citibike_df[['User Type ID', 'User Type']].drop_duplicates().sort_values(by='User Type ID').reset_index(drop=True)
users_df.columns = users_df.columns.str.lower().str.replace(' ', '_')
users_df.head()

,user_type_id,user_type
0,1,Subscriber
1,2,Customer


# 🛠️ Storing pandas DataFrames in PostgreSQL: Two Common Approaches

When working with `pandas` DataFrames, there are two primary methods to transfer data into a PostgreSQL database:

---

## 📁 **Approach 1: Export to CSV and Import via pgAdmin (Manual Method)**

Export a DataFrame to a `.csv` file and use **pgAdmin** to import it into a PostgreSQL table.

### ✅ Pros:
- Simple and beginner-friendly.
- Ideal for small datasets or one-time tasks.

### ❌ Cons:
- Manual and not scalable.
- Requires switching between Python and pgAdmin.
- Limited control over data types and schema.

---

## 🔗 **Approach 2: Connect to PostgreSQL and Write Tables Using SQLAlchemy (Automated Method)**

Use Python with the `SQLAlchemy` library to connect to PostgreSQL and write DataFrames directly to tables.

### ✅ Pros:
- Automated and scalable.
- Full control over schema and data types.
- Stays within Python, no tool-switching required.

### ❌ Cons:
- Requires setup (e.g., installing libraries, configuring connection).
- Slightly more complex for beginners.

---

## 📊 **Summary Table**

| **Feature**               | **Approach 1 (CSV + pgAdmin)** | **Approach 2 (SQLAlchemy)** |
|---------------------------|--------------------------------|-----------------------------|
| **Automation**            | Manual                        | Automated                  |
| **Scalability**           | Low                           | High                       |
| **Ease for Beginners**    | High                          | Moderate                   |
| **Control Over Schema**   | Limited                       | Full                       |
| **Tool Usage**            | Python + pgAdmin              | Python Only                |

Choose **Approach 1** for quick, small-scale tasks or if you're new to databases. Use **Approach 2** for larger datasets, automation, or production workflows.

## Approach 1

In [24]:
users_df.to_csv('../data/users.csv', index=False) 

## Approach 2

### Make the connection to the PostgreSQL instance

In [27]:
# Import necessary libraries for database connection
from sqlalchemy import create_engine, ForeignKey, text, Table, Column, Integer, String, MetaData, DateTime, Float, ForeignKeyConstraint
from sqlalchemy.schema import CreateTable

username = 'postgres'
password = '697597759'
host = 'localhost'
port = '5432'
database = 'bikerentals'

# SQLAlchemy engine creation
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

### Create a schema

In [18]:
# Create the schema if it doesn't exist
with engine.begin() as conn:  # begin() will auto-commit
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS bk;"))

### Creating Tables from DataFrames

#### Approach 1: Create Table via `to_sql`, Then Add Primary Key via SQL

This approach uses Pandas’ `to_sql()` method to automatically create the table based on the structure of the DataFrame and insert the data. After the table is created, a separate SQL command is executed to add a primary key constraint on the `"Station ID"` column.

**Pros:**
- Simple and quick for initial data loads.
- Lets Pandas handle the table creation without manually defining columns.
- Suitable for prototyping or ad-hoc work.

**Cons:**
- Adding a primary key afterward can fail if there are duplicate or null values.
- Relies on raw SQL for constraints, which may reduce portability.
- Less explicit schema definition — types are inferred from the DataFrame, which can lead to mismatches.


Stations

In [19]:
# Push DataFrame into the new schema
stations_df.to_sql('stations', engine, schema='bk', if_exists='replace', index=False)

# Define the stations table Primary Key
with engine.begin() as conn:
    conn.execute(text('ALTER TABLE bk.stations ADD PRIMARY KEY ("station_id");'))

Genders

In [22]:
gender_df.to_sql('genders', engine, schema='bk', if_exists='replace', index=False)

# Define the Genders table Primary Key
with engine.begin() as conn:
    conn.execute(text('ALTER TABLE bk.genders ADD PRIMARY KEY (gender_id);'))

Bikes

In [23]:
bikes_df.to_sql('bikes', engine, schema='bk', if_exists='replace', index=False)

# Define the Bikes table Primary Key
with engine.begin() as conn:
    conn.execute(text('ALTER TABLE bk.bikes ADD PRIMARY KEY (bike_id);'))

#### Approach 2: Define Table with SQLAlchemy, Then Load Data

This approach explicitly defines the table schema using SQLAlchemy's `Table` and `Column` objects, including primary key constraints. The table is created manually using `CreateTable`, and then `to_sql()` is used to insert data.

**Pros:**
- Full control over schema, data types, and constraints from the start.
- Safer and more robust — enforces schema validation before data is loaded.
- Better suited for production environments or complex database structures.

**Cons:**
- More verbose and requires knowledge of SQLAlchemy.
- `to_sql()` with `if_exists='replace'` will drop the table, which also removes the manually created constraints — use `append` to preserve schema.
- Slightly more complex setup, especially when syncing schema and data.

When using SQLAlchemy's ForeignKey('bk.stations.station_id'), SQLAlchemy tries to resolve the table bk.stations within its current MetaData() context. But since stations, bikes, and genders were created via to_sql() and not declared in SQLAlchemy, they don’t exist in metadata, and foreign key references to them fail.

In [38]:
metadata = MetaData()

trips_table = Table(
    'trips', metadata,
    Column('trip_id', Integer),
    Column('start_time', DateTime),  # Add other columns as needed
    Column('stop_time', DateTime),
    Column('trip_duration_minutes', Float),
    Column('trip_duration_seconds', Float),
    Column('start_station_id', Integer),
    Column('end_station_id', Integer),
    Column('bike_id', Integer),
    Column('user_type_id', Integer),
    Column('birth_year', Integer),
    Column('gender_id', Integer),
    schema='bk'
)

# Create the table manually
with engine.begin() as conn:
    conn.execute(CreateTable(trips_table, if_not_exists=True))
    
# Insert data into the stations table
trips_df.to_sql(
    'trips', engine,
    schema='bk',
    if_exists='replace',  # Use append in case of wanting to add data without dropping existing data
    index=False
)

with engine.begin() as conn:
    # Define the Primary Key for the trips table
    conn.execute(text('ALTER TABLE bk.trips ADD PRIMARY KEY (trip_id);'))
    # Add foreign key constraints
    conn.execute(
        text('ALTER TABLE bk.trips '
             'ADD CONSTRAINT fk_start_station '
             'FOREIGN KEY (start_station_id) REFERENCES bk.stations(station_id);')
    )
    conn.execute(
        text('ALTER TABLE bk.trips '
             'ADD CONSTRAINT fk_end_station '
             'FOREIGN KEY (end_station_id) REFERENCES bk.stations(station_id);')
    )
    conn.execute(
        text('ALTER TABLE bk.trips '
             'ADD CONSTRAINT fk_bike '
             'FOREIGN KEY (bike_id) REFERENCES bk.bikes(bike_id);')
    )
    conn.execute(
        text('ALTER TABLE bk.trips '
             'ADD CONSTRAINT fk_user_type '
             'FOREIGN KEY (user_type_id) REFERENCES bk.users(user_type_id);')
    )
    conn.execute(
        text('ALTER TABLE bk.trips '
             'ADD CONSTRAINT fk_gender '   
             'FOREIGN KEY (gender_id) REFERENCES bk.genders(gender_id);')
    )

In [40]:
# Verify Table in Schema
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM bk.stations;"))
    for row in result:
        print(row)

(3186, 'Grove St PATH', 40.71958611647166, -74.04311746358871)
(3209, 'Brunswick St', 40.7241765, -74.0506564)
(3195, 'Sip Ave', 40.73074262530658, -74.06378388404846)
(3211, 'Newark Ave', 40.72152515, -74.046304543)
(3187, 'Warren St', 40.7211236, -74.03805095)
(3183, 'Exchange Place', 40.7162469, -74.0334588)
(3213, 'Van Vorst Park', 40.71848892, -74.047726625)
(3193, 'Lincoln Park', 40.7246050998869, -74.07840594649315)
(3194, 'McGinley Square', 40.7253399253558, -74.06762212514877)
(3202, 'Newport PATH', 40.7272235, -74.0337589)
(3196, 'Riverview Park', 40.7443187, -74.0439909)
(3214, 'Essex Light Rail', 40.7127742, -74.0364857)
(3207, 'Oakland Ave', 40.7376037, -74.0524783)
(3199, 'Newport Pkwy', 40.7287448, -74.0321082)
(3203, 'Hamilton Park', 40.727595966, -74.044247311)
(3210, 'Pershing Field', 40.742677141, -74.051788633)
(3190, 'Garfield Ave Station', 40.7104670233797, -74.07003879547119)
(3185, 'City Hall', 40.7177325, -74.043845)
(3197, 'North St', 40.752559, -74.044725)
(3